# Assignment 5
### Submitted by Ankit Saini
#### 17BT30002

1) Download the MNIST handwritten digit dataset. It contains 28X28 images. Flatten them into 784-dimensional binary vectors. Keep aside 20% data for testing and another 20% for validation. [1 mark]

2) Now, draw a random subset of 10 dimensions (out of 784). Based on these 10 dimensions only, build a decision tree (using library function). Maximum depth allowed: 5. Calculate accuracy of the tree on validation set. [2 mark]

3) Repeat this process for 50 random subsets like this, each of dimension 10.For each of them, build a decision tree of max. depth 5. Calculate accuracy on validation set. [2 marks]

4) Carry out weighted classification of the test set using these 50 decision trees, along with their validation accuracies as weights. Report the accuracy. [1 marks]

5) Starting with this ensemble as the initial classifier, implement Adaboost algorithm. At each stage, build a decision tree using entropy based on weighted examples as the heterogeneity measure of each node. Each tree will have maximum depth of 5. Maximum 20 iterations of Adaboost. [3 marks]

6) Using this ensemble, carry out classification on the test set and report accuracy [1 mark]

#### import the requred libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import random
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
random.seed(42)

#### 1) Download the MNIST handwritten digit dataset. It contains 28X28 images. Flatten them into 784-dimensional binary vectors. Keep aside 20% data for testing and another 20% for validation. [1 mark]

In [2]:
train_path = "mnist_train.csv"
test_path = "mnist_test.csv"
validate_path = "mnist_validate.csv"
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
validate_df = pd.read_csv(validate_path)

In [3]:
train_df

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
y = train_df['label']
X = train_df[train_df.columns[1:]]

discretize the matrix

In [5]:
def discrete(x):
    x = x.T
    
    for i in x:
        mean = np.mean(x[i])
        a = []
        for j in x[i]:
            if j>mean:
                a.append(1)
            else:
                a.append(0)
        x[i] = a
    return x.T

In [6]:
X = discrete(X)

In [7]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

In [8]:
print('training data shape',X_train.shape, '  | 60% split of total dataframe')
print('validation data shape',X_val.shape, ' | 20% split of total dataframe')
print('testing data shape',X_test.shape, '    | 20% split of total dataframe')

training data shape (25200, 784)   | 60% split of total dataframe
validation data shape (8400, 784)  | 20% split of total dataframe
testing data shape (8400, 784)     | 20% split of total dataframe


#### 2) Now, draw a random subset of 10 dimensions (out of 784). Based on these 10 dimensions only, build a decision tree (using library function). Maximum depth allowed: 5. Calculate accuracy of the tree on validation set. [2 mark]

In [9]:
X_train_random_subset = X_train[X_train.columns.to_series().sample(10)]
X_val_random_subset = X_val[X_val.columns.to_series().sample(10)]
X_test_random_subset = X_test[X_test.columns.to_series().sample(10)]

In [10]:
clf = DecisionTreeClassifier(max_depth = 5, random_state = 42).fit(X_train_random_subset, y_train)
y_val_pred = clf.predict(X_val_random_subset)
y_test_pred = clf.predict(X_test_random_subset)

Accuracy over validation dataset

In [11]:
print(accuracy_score(y_val,y_val_pred))

0.17964285714285713


Accuracy over test dataset

In [12]:
print(accuracy_score(y_test,y_test_pred))

0.08535714285714285


#### 3) Repeat this process for 50 random subsets like this, each of dimension 10.For each of them, build a decision tree of max. depth 5. Calculate accuracy on validation set. [2 marks]

In [13]:
skf = StratifiedKFold(n_splits=50, random_state=42)

trees = []

for i in range(50):
    j = random.randint(1,8000)
    clf = DecisionTreeClassifier(max_depth = 5, random_state = 42).fit(X_train_random_subset[0:j+10], y_train[0:j+10])
    trees.append(clf)
    X_val_random_subset1 = X_val_random_subset[j:j+10]
    y_val1 = y_val[j:j+10]
    y_val_pred1 = clf.predict(X_val_random_subset1)
    print('validation sample #',i+1)
    print(accuracy_score(y_val1,y_val_pred1))
    
    


validation sample # 1
0.0
validation sample # 2
0.2
validation sample # 3
0.2
validation sample # 4
0.1
validation sample # 5
0.2
validation sample # 6
0.2
validation sample # 7
0.2
validation sample # 8
0.3
validation sample # 9
0.1
validation sample # 10
0.0
validation sample # 11
0.2
validation sample # 12
0.2
validation sample # 13
0.1
validation sample # 14
0.0
validation sample # 15
0.3
validation sample # 16
0.0
validation sample # 17
0.1
validation sample # 18
0.1
validation sample # 19
0.1
validation sample # 20
0.2
validation sample # 21
0.1
validation sample # 22
0.2
validation sample # 23
0.4
validation sample # 24
0.2
validation sample # 25
0.2
validation sample # 26
0.4
validation sample # 27
0.4


C:\Users\SARANSH\Anaconda3\envs\saransh\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


validation sample # 28
0.1
validation sample # 29
0.0
validation sample # 30
0.1
validation sample # 31
0.2
validation sample # 32
0.2
validation sample # 33
0.2
validation sample # 34
0.1
validation sample # 35
0.2
validation sample # 36
0.0
validation sample # 37
0.3
validation sample # 38
0.2
validation sample # 39
0.0
validation sample # 40
0.1
validation sample # 41
0.2
validation sample # 42
0.3
validation sample # 43
0.2
validation sample # 44
0.1
validation sample # 45
0.1
validation sample # 46
0.0
validation sample # 47
0.2
validation sample # 48
0.1
validation sample # 49
0.3
validation sample # 50
0.2


#### 4) Carry out weighted classification of the test set using these 50 decision trees, along with their validation accuracies as weights. Report the accuracy. [1 marks]

In [14]:
def weighted_classification(acc , y_val_pre):
    
    d = []
    for i in range(50):
        d.append(y_val_pre[i]*acc[i])
        
    b = []
    for j in range(len(y_val_pre[0])):
        for i in range(50):
            a = 0
            a+=d[i][j]
        b.append(int(a))
        
        
    return b  

In [15]:
acc = []
y_val_pre = []
for w_classifier in trees:
    yy = w_classifier.predict(X_val_random_subset)
    y_val_pre.append(yy)
    acc.append(accuracy_score(y_val,yy))

Accuracy after weighted classification is:

In [16]:
accuracy_score(y_val,weighted_classification(acc , y_val_pre))

0.11916666666666667

#### 5) Starting with this ensemble as the initial classifier, implement Adaboost algorithm. At each stage, build a decision tree using entropy based on weighted examples as the heterogeneity measure of each node. Each tree will have maximum depth of 5. Maximum 20 iterations of Adaboost. [3 marks]

In [17]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_hastie_10_2
import matplotlib.pyplot as plt

# HELPER FUNCTION: GET ERROR RATE
def get_error_rate(pred, Y):
    return sum(pred != Y) / float(len(Y))

# HELPER FUNCTION: PRINT ERROR RATE
def print_error_rate(err):
    print ('Error rate: Training: %.4f - Test: %.4f' % err)

# HELPER FUNCTION: GENERIC CLASSIFIER
def generic_clf(y_train, X_train, y_test, X_test, clf):
    clf.fit(X_train,y_train)
    pred_train = clf.predict(X_train)
    pred_test = clf.predict(X_test)
    return get_error_rate(pred_train, y_train), \
           get_error_rate(pred_test, y_test)

# ADABOOST IMPLEMENTATION
def adaboost_clf(y_train, X_train, y_test, X_test, M, clf):
    n_train, n_test = len(X_train), len(X_test)
    # Initialize weights
    w = np.ones(n_train) / n_train
    pred_train, pred_test = [np.zeros(n_train), np.zeros(n_test)]
    
    for i in range(20):
        print('iteration #', i+1)
        # Fit a classifier with the specific weights
        clf.fit(X_train, y_train, sample_weight = w)
        pred_train_i = clf.predict(X_train)
        pred_test_i = clf.predict(X_test)
        # Indicator function
        miss = [int(x) for x in (pred_train_i != y_train)]
        # Equivalent with 1/-1 to update weights
        miss2 = [x if x==1 else -1 for x in miss]
        # Error
        err_m = np.dot(w,miss) / sum(w)
        # Alpha
        alpha_m = 0.5 * np.log( (1 - err_m) / float(err_m))
        # New weights
        w = np.multiply(w, np.exp([float(x) * alpha_m for x in miss2]))
        # Add to prediction
        pred_train = [sum(x) for x in zip(pred_train, 
                                          [x * alpha_m for x in pred_train_i])]
        pred_test = [sum(x) for x in zip(pred_test, 
                                         [x * alpha_m for x in pred_test_i])]
    
    pred_train, pred_test = np.sign(pred_train), np.sign(pred_test)
    
    return pred_test
    # Return error rate in train and test set
#     return get_error_rate(pred_train, y_train), \
#            get_error_rate(pred_test, y_test)

#Plot Function
def plot_error_rate(er_train, er_test):
    df_error = pd.DataFrame([er_train, er_test]).T
    df_error.columns = ['Training', 'Test']
    plot1 = df_error.plot(linewidth = 3, figsize = (8,6),
            color = ['lightblue', 'darkblue'], grid = True)
    plot1.set_xlabel('Number of iterations', fontsize = 12)
    plot1.set_xticklabels(range(0,450,50))
    plot1.set_ylabel('Error rate', fontsize = 12)
    plot1.set_title('Error rate vs number of iterations', fontsize = 16)
    plt.axhline(y=er_test[0], linewidth=1, color = 'red', ls = 'dashed')

if __name__ == '__main__':
    
    # Fit a simple decision tree first
    clf_tree = DecisionTreeClassifier(max_depth = 5, random_state = 42)
    er_tree = generic_clf(y_train, X_train, y_test, X_test, clf_tree)
    
    # Fit Adaboost classifier using a decision tree as base estimator
    # Test with different number of iterations
    er_train, er_test = [er_tree[0]], [er_tree[1]]
    x_range = range(10, 200, 10)
    y_pred = adaboost_clf(y_train, X_train, y_val, X_val, 0, clf_tree)
    
    print(accuracy_score(y_val,y_pred))

iteration # 1
iteration # 2
iteration # 3
iteration # 4
iteration # 5
iteration # 6
iteration # 7
iteration # 8
iteration # 9
iteration # 10
iteration # 11
iteration # 12
iteration # 13
iteration # 14
iteration # 15
iteration # 16
iteration # 17
iteration # 18
iteration # 19
iteration # 20
0.14547619047619048


#### 6) Using this ensemble, carry out classification on the test set and report accuracy [1 mark]

In [18]:
if __name__ == '__main__':
    
    
    # Fit a simple decision tree first
    clf_tree = DecisionTreeClassifier(max_depth = 5, random_state = 42)
    er_tree = generic_clf(y_train, X_train, y_test, X_test, clf_tree)
    
    
    # Fit Adaboost classifier using a decision tree as base estimator
    # Test with different number of iterations
    er_train, er_test = [er_tree[0]], [er_tree[1]]
    x_range = range(10, 200, 10)
    er_i = adaboost_clf(y_train, X_train, y_test, X_test, 0, clf_tree)
    print(accuracy_score(y_test,er_i))


iteration # 1
iteration # 2
iteration # 3
iteration # 4
iteration # 5
iteration # 6
iteration # 7
iteration # 8
iteration # 9
iteration # 10
iteration # 11
iteration # 12
iteration # 13
iteration # 14
iteration # 15
iteration # 16
iteration # 17
iteration # 18
iteration # 19
iteration # 20
0.14523809523809525
